# Description

Print out all the links to the published notebooks stored in the given text file.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os

import pandas as pd
from IPython.display import HTML, display

import core.config as cconfig
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import oms.broker.ccxt.ccxt_logger as obcccclo
import oms.execution_analysis_configs as oexancon

/app/amp/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)
_LOG = logging.getLogger(__name__)
_LOG.info("%s", henv.get_system_signature()[0])
hprint.config_notebook()

INFO  > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-32ed6b6f-d96c-4319-af8e-6703b856aab5.json'
INFO  # Git
  branch_name='CmTask8300_fix_post_run_balances'
  hash='7134a6a38'
  # Last commits:
    *   7134a6a38 Nina Lee Merge branch 'master' into CmTask8300_fix_post_run_balances       (   3 hours ago) Mon Jun 3 11:59:46 2024  (HEAD -> CmTask8300_fix_post_run_balances, origin/CmTask8300_fix_post_run_balances)
    |\  
    | * 3cce8bf2b pavolrabatin CmTask8421_Add_aws_efs_access_point_Resource_to_the_EFS_Module (#8465) (   5 hours ago) Mon Jun 3 10:15:39 2024  (origin/CmampTask8483_Improve_shadow_trading_website_for_investors)
    | * 6abdbbfc7 Henry Wu CmTask8414_Get_Info_From_Folkapp (#8430)                          (    2 days ago) Sat Jun 1 16:22:00 2024           
# Machine info
  system=Linux
  node name=260429f2eaf6
  release=5.15.0-1056-aws
  version=#61~20.04.1-Ubuntu SMP Wed Mar 13 17:40:41 UTC 2024
  machine=x86_64


# Config

In [4]:
config = cconfig.get_config_from_env()
if config:
    _LOG.info("Using config from env vars")
else:
    system_log_dir = "/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/system_log_dir.manual"
    timestamp_dir, _ = os.path.split(system_log_dir)
    analysis_notebooks_file_path = os.path.join(
        system_log_dir,
        "analysis_notebooks",
        "analysis_notebooks_links.csv",
    )
    config_list = oexancon.get_master_trading_system_report_notebook_config(
        timestamp_dir, analysis_notebooks_file_path
    )
    config = config_list[0]
print(config)

timestamp_dir: /shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000
analysis_notebooks_file_path: /shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/system_log_dir.manual/analysis_notebooks/analysis_notebooks_links.csv


# Utils

In [5]:
# TODO(Vlad): The same as in the `Master_buildmeister_dashboard`. Have to be moved to the helpers.
# TODO(Toma): Move functions to `oms/notebooks/notebooks_utils.py`.
def make_clickable(url: str) -> str:
    """
    Wrapper to make the URL value clickable.

    :param url: URL value to convert
    :return: clickable URL link
    """
    return f'<a href="{url}" target="_blank">{url}</a>'


def get_balances(ccxt_log: obcccclo.CcxtLogger) -> pd.DataFrame:
    """
    Load balances Dataframe.
    """
    balances = ccxt_log.load_balances()
    # Get the balances JSON from the list returned by `load_balances` and
    # transform per-asset information to Dataframe.
    # Pick the latest balance to process the edge case when the data from
    # multiple runs is saved to the same directory due to re-runs.
    if len(balances) > 1:
        # TODO(Toma): consider assert.
        _LOG.info("Multiple files found, using the balances from the latest run")
    balances_df = pd.DataFrame(balances[-1]["info"]["assets"])
    balances_df = balances_df[["asset", "walletBalance"]]
    return balances_df

# Print the notebooks links

In [6]:
df = pd.read_csv(config["analysis_notebooks_file_path"])
df["URL"] = df["URL"].apply(make_clickable)
display(HTML(df.to_html(escape=False)))

,Notebook Name,URL
0,Master_system_reconciliation_fast,http://172.30.2.44/CmTask7895/Master_system_reconciliation_fast.0.20240509-131036.html
1,Master_broker_portfolio_reconciliation,http://172.30.2.44/CmTask7895/Master_broker_portfolio_reconciliation.0.20240513-130258.html
2,Master_broker_debugging,http://172.30.2.44/CmTask7895/Master_broker_debugging.0.20240513-130434.html
3,Master_execution_analysis,http://172.30.2.44/CmTask7895/Master_execution_analysis.0.20240513-130641.html
4,Master_bid_ask_execution_analysis,http://172.30.2.44/CmTask7895/Master_bid_ask_execution_analysis.0.20240513-131524.html


# Load balances

## Before run

In [7]:
# Init the log reader.
log_dir = os.path.join(config["timestamp_dir"], "flatten_account.before")
ccxt_log_reader = obcccclo.CcxtLogger(log_dir)

WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/broker_config.json' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/args.json' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/child_order_fills' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/ccxt_child_order_responses' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/oms_child_orders' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.before/oms_parent_orders' doesn't exist!
WARNING Path '/shared_data/CmTask

In [8]:
balances_df = get_balances(ccxt_log_reader)
display(balances_df)

Loading '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_acc…

,asset,walletBalance
0,FDUSD,0.00000000
1,BTC,0.00000000
2,XRP,0.00000000
3,TUSD,0.00000000
4,BNB,0.00000000
5,ETH,0.00000000
6,USDT,8574.04872986
7,USDC,0.00000000


## After run

In [9]:
# Init the log reader.
log_dir = os.path.join(config["timestamp_dir"], "flatten_account.after")
ccxt_log_reader = obcccclo.CcxtLogger(log_dir)

WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/broker_config.json' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/args.json' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/child_order_fills' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/ccxt_child_order_responses' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/oms_child_orders' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_account.after/oms_parent_orders' doesn't exist!
WARNING Path '/shared_data/CmTask7852_5

In [10]:
balances_df = get_balances(ccxt_log_reader)
display(balances_df)

Loading '/shared_data/CmTask7852_5/system_reconciliation/C11a/prod/20240509_155500.20240509_165000/flatten_acc…

,asset,walletBalance
0,FDUSD,0.00000000
1,BTC,0.00000000
2,XRP,0.00000000
3,TUSD,0.00000000
4,BNB,0.00000000
5,ETH,0.00000000
6,USDT,8570.33015946
7,USDC,0.00000000
